In [1]:
#取得歷史資料,將資料做簡單處理
Match_data1 = [ line.strip('\n').split(",") for line in open('TXFB7.I020') ]
Match_data2 = [ line.strip('\n').split(",") for line in open('TXFB7.I080') ][1:]
#取得歷史日期
Date_list1 = set([ i[0] for i in Match_data1 ])
#Date_list.remove(' ')
Date_list1=sorted(list(Date_list1))
Date_list2 = set([ i[0] for i in Match_data2 ])
#Date_list.remove(' ')
Date_list2=sorted(list(Date_list2))

In [2]:
#開高低收量function(輸入-->'日期',起始時間,結束時間  輸出-->開高低收量)
def OHLC(DD,TT1,TT2):
#取得回測每日資料
    C_I020 = [ i for i in Match_data1 if i[0]== DD ]
#截取時間段資料並存入陣列
    I020a= [ line for line in C_I020 if int(line[1])> TT1 and int(line[1])< TT2 ]
#第一筆的資料價格
    O= I020a[0][3]
#最後一筆資料的價格
    C= I020a[-1][3]
    x1=[]
    x2=[]
    for i in range(len(I020a)):
#將所有價格存為價格陣列
        x1.append(int(I020a[i][3]))
#將所有量存為量陣列
        x2.append(int(I020a[i][4]))
#取出最高價
    H=max(x1)
#取出最低價
    L=min(x1)
#算出總量
    Amount=sum(x2)
    return [O,H,L,C,Amount]

#價格function(輸入-->日期時間  輸出-->價格)
def GP(date,time):
#取得回測每日資料
 C_I020 = [ i for i in Match_data1 if i[0]== date ]    
#截取時間段資料並存入陣列
 I020a= [ line for line in C_I020 if int(line[1])<=time ]
#取出最後一筆(最接近時間點)的價格與時間 
 OrderTime=I020a[-1][1]  #下單時間紀錄
 OrderPrice=int(I020a[-1][3]) #下單價格紀錄
 return [OrderTime,OrderPrice]

In [43]:
#計算每日盤中外盤比
def GetOutDeskRatio(DD):
    InDesk=0
    OutDesk=0
    OutDeskRatio=0
#取得回測每日資料
    C_I020 = [ i for i in Match_data1 if i[0]== DD ]
#截取時間段資料並存入陣列
    I020a= [ line for line in C_I020 if int(line[1])> 8450000 and int(line[1])< 10300000 ]
    
#取得回測每日資料
    C_I080 = [ i for i in Match_data2 if i[0]== DD ]
#截取時間段資料並存入陣列
    I080a= [ line for line in C_I080 if int(line[1])> 8450000 and int(line[1])< 10300000 ]
    num1=0
    num2=0
    while num1 != len(I020a):
        if int(I020a[num1][1]) == int(I080a[num2][1]):
            if int(I020a[num1][3])>((int(I080a[num2][3])+int(I080a[num2][13]))*0.5):
                OutDesk+=int(I020a[num1][4])
                num1+=1
            elif int(I020a[num1][3])<((int(I080a[num2][3])+int(I080a[num2][13]))*0.5):
                InDesk+=int(I020a[num1][4])
                num1+=1
            else:
                num1+=1
        else:
            num2+=1

    OutDeskRatio=OutDesk/(OutDesk+InDesk)*100
    return OutDeskRatio
GetOutDeskRatio('20170119')
    

48.66863566636903

In [38]:
#計算對應到的理論漲跌幅
def OutDeskValue(DD):
    value={35:-2.69,36:-2.51,37:-2.33,38:-2.15,39:-1.97,40:-1.79,41:-1.61,42:-1.43,43:-1.25,44:-1.07,45:-0.89,46:-0.71,47:-0.54,48:-0.36,49:-0.18,50:0.00,51:0.18,52:0.36,53:0.54,54:0.72,55:0.90,56:1.08,57:1.26,58:1.44,59:1.61,60:1.79,61:1.97,62:2.15,63:2.33,64:2.51,65:2.69}
    if GetOutDeskRatio(DD)<=35:
        return value[35]
    elif GetOutDeskRatio(DD)>65:
        return value[65]
    else:
        return value[int(GetOutDeskRatio(DD))]

In [41]:
OutDeskValue('20170119')

-0.36

In [4]:
#每日回測
profit=[]
for i in range(len(Date_list1)):
    DD='Date_list1[i]'
    #計算當前漲跌幅=(1030收盤減開盤)/(開盤)
    CurrentSpread=((OHLC(DD,8450000,10300000)[0])-(OHLC(DD,8450000,10300000)[3]))/(OHLC(DD,8450000,10300000)[0])*100

    #確認買進訊號
    BuySignal=0
    SellShortSignal=0
    if int(OutDeskValue(DD))>CurrentSpread+1: #當理論值大於實際值+1即買進
        BuySignal=1
        
    elif int(OutDeskValue(DD))<CurrentSpread-1: #當理論值小於實際值-1即賣出
        SellShortSignal=1
    else:
        BuySignal=0
        SellShortSignal=0 

   
    #取得回測每日資料
    C_I020 = [ i for i in Match_data1 if i[0]== Date_list1[i] ]
    
    #設立買賣時間
    I020a= [ line for line in C_I020 if int(line[1])>=10300000 and int(line[1])<=12300000 ]
    I020b= [ line for line in C_I020 if int(line[1])>=8450000 and int(line[1])<=13300000 ]
    #當時間=1030時 確認是否有進場訊號
    for i in range(1,len(I020a)):
        if I020a[1+i][1]==10300000  and BuySignal==1:
            OrderDay=I020a[i][0]  
            OrderTime=I020a[i][1] 
            OrderPrice=int(I020a[i][3])              
        elif I020a[1+i][1]==10300000  and SellShortSignal==1:
            OrdershortDay=I020a[i][0]  
            OrderTime=I020a[i][1]  
            OrderPrice=int(I020a[i][3])     
        print(BuyDay)
            
    for i in range(1,len(I020b)):
        price=int(I020a[i][3])
        if price <= int(OrderPrice)-30 or price >= int(OrderPrice)+30:
            CoverDay=I020b[i][0]  #平倉日
            CoverTime=I020b[i][1]  #平倉時間紀錄
            CoverPrice=int(I020b[i][3])  #平倉時間紀錄
            value=CoverPrice-OrderPrice
            break;
        elif i == len(I020a)-1:
            CoverDay=I020b[i][0]  #平倉日
            CoverTime=I020b[i][1]  #平倉時間紀錄
            CoverPrice=int(I020b[i][3]) #平倉時間紀錄
            value=CoverPrice-OrderPrice
    profit.append(value)
    #print("Date:",CoverDay," OrderTime:",," OrderPrice:",op[1]," CoverTime:",CoverTime," CoverPrice:",CoverPrice," Profit:",CoverPrice-op[1])


TypeError: list indices must be integers or slices, not list

In [66]:
    DD='Date_list1[0]'
    #計算當前漲跌幅=(1030收盤減開盤)/(開盤)
    CurrentSpread=((OHLC(DD,8450000,10300000)[0])-(DD,8450000,10300000)[3]))/(int(OHLC(Match_data1[0][0]),8450000,10300000)[0])*100

IndexError: list index out of range

In [ ]:
#每日回測
for DateNum in range(len(Date_list1)):
    DD='Date_list1[DateNum]'
    #計算當前漲跌幅=(1030收盤減開盤)/(開盤)
    CurrentSpread=((OHLC(DD,8450000,10300000)[0])-(OHLC(DD,8450000,10300000)[3]))/(OHLC(DD,8450000,10300000)[0])*100

    #確認買進訊號
    BuySignal=0
    SellShortSignal=0
    if int(OutDeskValue(DD))>CurrentSpread+1: #當理論值大於實際值+1即買進
        BuySignal=1
        
    elif int(OutDeskValue(DD))<CurrentSpread-1: #當理論值小於實際值-1即賣出
        SellShortSignal=1
    else:
        BuySignal=0
        SellShortSignal=0    

   
    #取得回測每日資料
    C_I020 = [ i for i in Match_data1 if i[0]== Date_list[DateNum] ]
    
    #設立買賣時間
    I020a= [ line for line in C_I020 if int(line[1])>=10300000 and int(line[1])<=12300000 ]
    I020b= [ line for line in C_I020 if int(line[1])>=8450000 and int(line[1])<=13300000 ]
    #當時間=1030時 確認是否有進場訊號
    for i in range(1,len(I020a)):
        if I020a[1+i][1]==10300000  and BuySignal==1:
            OrderDay=I020a[i][0]  
            OrderTime=I020a[i][1] 
            OrderPrice=int(I020a[i][3])              
        elif I020a[1+i][1]==10300000  and SellShortSignal==1:
            OrdershortDay=I020a[i][0]  
            OrderTime=I020a[i][1]  
            OrderPrice=int(I020a[i][3])     
        print(BuyDay)
            
    for i in range(1,len(I020b)):
        price=int(I020a[i][3])
        if price <= int(OrderPrice)-30 or price >= int(OrderPrice)+30:
            CoverDay=I020b[i][0]  #平倉日
            CoverTime=I020b[i][1]  #平倉時間紀錄
            CoverPrice=int(I020b[i][3])  #平倉時間紀錄
            break;
        elif i == len(I020a)-1:
            CoverDay=I020b[i][0]  #平倉日
            CoverTime=I020b[i][1]  #平倉時間紀錄
            CoverPrice=int(I020b[i][3]) #平倉時間紀錄
    #print("Date:",CoverDay," OrderTime:",," OrderPrice:",op[1]," CoverTime:",CoverTime," CoverPrice:",CoverPrice," Profit:",CoverPrice-op[1])
